<a href="https://colab.research.google.com/github/cjvilla/American-Wildfires/blob/main/exploratory_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Modules**

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
raw_data = pd.read_csv('../content/drive/MyDrive/eNose/sensor_data.csv')

In [28]:
raw_data.columns

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11',
       'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21',
       'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31',
       'S32', 'real_idn', 'group'],
      dtype='object')

# **ETL/Data Cleaning**

In [29]:
demographics = pd.read_csv("/content/drive/MyDrive/eNose/demographics.csv")

In [30]:
# Define a function to map values in the 'Sex' column to 'Sex_Description'
def map_sex_to_description(sex_value):
    if sex_value == 1:
        return 'Female'
    elif sex_value == 2:
        return 'Male'
    else:
        return 'Unknown'  # You can specify a default value for other cases if needed

# Update the 'Sex_Description' column based on the 'Sex' column
demographics['Sex_Description'] = np.vectorize(map_sex_to_description)(demographics['Sex'])


In [31]:
# Define a function to map values in the 'Race' column to 'Race_Description'
def map_race_to_description(race_value):
    if race_value == 1:
        return 'Asian'
    elif race_value == 2:
        return 'Black'
    elif race_value == 3:
        return 'Hispanic'
    elif race_value == 4:
        return 'White'
    else:
        return 'Unknown'  # You can specify a default value for other cases if needed

# Apply the mapping function to create the new column
demographics['Race_Description'] = np.vectorize(map_race_to_description)(demographics['Race'])


In [32]:
# Define a function to map values in the 'Group' column to 'Group_Diagnosis'
def map_group_to_diagnosis(group_value):
    if group_value == 0:
        return 'Healthy Control'
    elif group_value == 1:
        return 'Alcoholic Liver Disease'
    elif group_value == 2:
        return "Parkinson's"
    else:
        return 'Unknown'  # You can specify a default value for other cases if needed

# Apply the mapping function to create the new column
demographics['Group_Diagnosis'] = np.vectorize(map_group_to_diagnosis)(demographics['group'])


In [33]:
# Define the desired column order
desired_column_order = [
    'real_idn', 'Age', 'Sex', 'Sex_Description', 'Race', 'Race_Description', 'BMI',
    'LPS_Endotoxin', 'Perm_sucralose', 'group', 'Group_Diagnosis', 'S1_mean', 'S2_mean',
    'S3_mean', 'S4_mean', 'S5_mean', 'S6_mean', 'S7_mean', 'S8_mean', 'S9_mean',
    'S10_mean', 'S11_mean', 'S12_mean', 'S13_mean', 'S14_mean', 'S15_mean', 'S16_mean',
    'S17_mean', 'S18_mean', 'S19_mean', 'S20_mean', 'S21_mean', 'S22_mean', 'S23_mean',
    'S24_mean', 'S25_mean', 'S26_mean', 'S27_mean', 'S28_mean', 'S29_mean', 'S30_mean',
    'S31_mean', 'S32_mean'
]

# Reorder the columns
demographics = demographics[desired_column_order]


In [34]:
# merging the demographic data columns with sensors of the raw data
merged_df = pd.merge(demographics, raw_data, on=['group', 'real_idn'], how='inner')


In [35]:

# Get a list of columns to drop based on the condition
columns_to_drop = [col for col in merged_df.columns if '_mean' in col]

# Drop the columns from the DataFrame
merged_df = merged_df.drop(columns=columns_to_drop)

In [36]:
merged_df.columns

Index(['real_idn', 'Age', 'Sex', 'Sex_Description', 'Race', 'Race_Description',
       'BMI', 'LPS_Endotoxin', 'Perm_sucralose', 'group', 'Group_Diagnosis',
       'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11',
       'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21',
       'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31',
       'S32'],
      dtype='object')

# **Testing and Training**

# TCN

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your merged_dfset and preprocess it
# Replace 'your_merged_dfset.csv' with your merged_dfset file path

# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
          'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
          'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 1 for Alcoholic Liver Disease and 2 for Parkinson's

# Split the merged_dfset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input data to include timesteps
# Assuming each sensor reading is treated as a separate timestep
X_train = X_train[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]

# Create a TCN model
def create_tcn_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
    ])
    return model

# Initialize and compile the model
input_shape = (X_train.shape[1], X_train.shape[2])
tcn_model = create_tcn_model(input_shape)
tcn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the TCN model
epochs = 10
batch_size = 32
tcn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = tcn_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/10
139/139 [==============================] - 2s 8ms/step - loss: -2.4251 - accuracy: 0.3157 - val_loss: -14.8935 - val_accuracy: 0.2698
Epoch 2/10
139/139 [==============================] - 1s 6ms/step - loss: -117.7198 - accuracy: 0.4081 - val_loss: -451.4483 - val_accuracy: 0.3813
Epoch 3/10
139/139 [==============================] - 1s 6ms/step - loss: -986.3389 - accuracy: 0.4160 - val_loss: -2164.3730 - val_accuracy: 0.4199
Epoch 4/10
139/139 [==============================] - 1s 6ms/step - loss: -3926.8401 - accuracy: 0.4034 - val_loss: -7425.9736 - val_accuracy: 0.4422
Epoch 5/10
139/139 [==============================] - 1s 6ms/step - loss: -10921.8906 - accuracy: 0.4099 - val_loss: -19250.3809 - val_accuracy: 0.5700
Epoch 6/10
139/139 [==============================] - 1s 6ms/step - loss: -24184.4023 - accuracy: 0.4099 - val_loss: -37693.2578 - val_accuracy: 0.3834
Epoch 7/10
139/139 [==============================] - 1s 9ms/step - loss: -46109.3945 - accuracy: 0.4059

## Random Forest

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your merged_dfset and preprocess it (as you've done)
# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
          'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
          'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 0 for Healthy Control and 1 for Alcoholic Liver Disease

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_classes = [0, 1]
X_filtered = X[y.isin(selected_classes)]
y_filtered = y[y.isin(selected_classes)]

# Split the filtered dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

# Standardize the input features (important for Random Forest)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Predict using the trained Random Forest classifier
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 93.10%


In [37]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Load your merged_df dataset and preprocess it
# Replace 'your_merged_df.csv' with your dataset file path

# Create an empty list to store DataFrames
results_list = []

# Define features (X) and target variable (y) for classification
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # 0 for Healthy Control, 1 for Alcoholic Liver Disease, and 2 for Parkinson's

# Specify the number of iterations
num_iterations = 32

for iteration in range(num_iterations):
    # Split the dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier with specific hyperparameters
    xgb_classifier = xgb.XGBClassifier(
        objective='binary:logistic',
        random_state=42,
        n_estimators=100,  # Number of boosting rounds
        max_depth=6,       # Maximum tree depth
        learning_rate=0.1, # Learning rate
        subsample=0.8,     # Fraction of samples used for each tree
    )
    xgb_classifier.fit(X_train, y_train)

    # Predict disease labels on the test set
    y_pred = xgb_classifier.predict(X_test)

    # Calculate accuracy for each class
    accuracy_healthy_control = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
    accuracy_alcoholic_liver_disease = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])
    accuracy_parkinsons = accuracy_score(y_test[y_test == 2], y_pred[y_test == 2])

    # Create a DataFrame for the current iteration with model information
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Healthy Control Accuracy': [accuracy_healthy_control],
        'Alcoholic Liver Disease Accuracy': [accuracy_alcoholic_liver_disease],
        'Parkinson\'s Accuracy': [accuracy_parkinsons],
        'Model Info': [str(xgb_classifier)],
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_classification_results = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_classification_results.head(20)


,Iteration,Healthy Control Accuracy,Alcoholic Liver Disease Accuracy,Parkinson's Accuracy,Model Info
0,1,0.905473,0.820588,0.726644,"XGBClassifier(base_score=None, booster=None, c..."
1,2,0.898276,0.833819,0.676375,"XGBClassifier(base_score=None, booster=None, c..."
2,3,0.900974,0.800000,0.690722,"XGBClassifier(base_score=None, booster=None, c..."
3,4,0.911223,0.803519,0.704082,"XGBClassifier(base_score=None, booster=None, c..."
4,5,0.909710,0.850299,0.659164,"XGBClassifier(base_score=None, booster=None, c..."
5,6,0.898058,0.814815,0.627760,"XGBClassifier(base_score=None, booster=None, c..."
6,7,0.925801,0.827485,0.676768,"XGBClassifier(base_score=None, booster=None, c..."
7,8,0.899489,0.802469,0.697819,"XGBClassifier(base_score=None, booster=None, c..."
8,9,0.917763,0.849530,0.662295,"XGBClassifier(base_score=None, booster=None, c..."
9,10,0.925424,0.857143,0.608563,"XGBClassifier(base_score=None, booster=None, c..."


## xgBoost

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Load your merged_dfset and preprocess it (as you've done)
# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
          'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
          'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 0 for Healthy Control and 1 for Alcoholic Liver Disease

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_classes = [0, 1]
X_filtered = X[y.isin(selected_classes)]
y_filtered = y[y.isin(selected_classes)]

# Split the filtered dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

# Standardize the input features (important for XGBoost)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_classifier.fit(X_train, y_train)

# Predict using the trained XGBoost classifier
y_pred = xgb_classifier.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 93.53%


## Neural Network (Tensorflow)

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score

# Load your merged_df dataset and preprocess it (as you've done)
# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
          'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
          'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 0 for Healthy Control and 1 for Alcoholic Liver Disease

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_classes = [0, 1]
X_filtered = X[y.isin(selected_classes)]
y_filtered = y[y.isin(selected_classes)]

# Split the filtered dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

# Standardize the input features (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a simple neural network model
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    keras.layers.Dense(64, activation='relu'),     # Hidden layer with 64 neurons and ReLU activation
    keras.layers.Dense(1, activation='sigmoid')    # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the neural network on your data
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/20
93/93 [==============================] - 1s 4ms/step - loss: 0.4837 - accuracy: 0.8022 - val_loss: 0.4376 - val_accuracy: 0.8304
Epoch 2/20
93/93 [==============================] - 0s 3ms/step - loss: 0.4323 - accuracy: 0.8218 - val_loss: 0.4096 - val_accuracy: 0.8358
Epoch 3/20
93/93 [==============================] - 0s 3ms/step - loss: 0.4101 - accuracy: 0.8359 - val_loss: 0.3873 - val_accuracy: 0.8371
Epoch 4/20
93/93 [==============================] - 0s 3ms/step - loss: 0.3917 - accuracy: 0.8447 - val_loss: 0.3812 - val_accuracy: 0.8385
Epoch 5/20
93/93 [==============================] - 0s 3ms/step - loss: 0.3801 - accuracy: 0.8487 - val_loss: 0.3661 - val_accuracy: 0.8398
Epoch 6/20
93/93 [==============================] - 0s 2ms/step - loss: 0.3684 - accuracy: 0.8494 - val_loss: 0.3589 - val_accuracy: 0.8398
Epoch 7/20
93/93 [==============================] - 0s 2ms/step - loss: 0.3613 - accuracy: 0.8467 - val_loss: 0.3533 - val_accuracy: 0.8398
Epoch 8/20
93/93 [==

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score

# Load your merged_df dataset and preprocess it (as you've done)
# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 0 for Healthy Control and 1 for Alcoholic Liver Disease

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_classes = [0, 1]
X_filtered = X[y.isin(selected_classes)]
y_filtered = y[y.isin(selected_classes)]

# Split the filtered dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

# Standardize the input features (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for LSTM (assuming time steps)
n_features = X_train.shape[1]
X_train = X_train.reshape(-1, 1, n_features)
X_test = X_test.reshape(-1, 1, n_features)

# Create an LSTM-based model
model = keras.Sequential([
    keras.layers.LSTM(64, activation='relu', input_shape=(1, n_features)),
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the LSTM model on your data
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/20
93/93 [==============================] - 3s 9ms/step - loss: 0.5208 - accuracy: 0.8005 - val_loss: 0.4589 - val_accuracy: 0.8345
Epoch 2/20
93/93 [==============================] - 0s 5ms/step - loss: 0.4548 - accuracy: 0.8208 - val_loss: 0.4268 - val_accuracy: 0.8358
Epoch 3/20
93/93 [==============================] - 0s 3ms/step - loss: 0.4290 - accuracy: 0.8238 - val_loss: 0.4053 - val_accuracy: 0.8358
Epoch 4/20
93/93 [==============================] - 0s 3ms/step - loss: 0.4056 - accuracy: 0.8322 - val_loss: 0.3884 - val_accuracy: 0.8385
Epoch 5/20
93/93 [==============================] - 0s 3ms/step - loss: 0.3791 - accuracy: 0.8450 - val_loss: 0.3617 - val_accuracy: 0.8318
Epoch 6/20
93/93 [==============================] - 0s 3ms/step - loss: 0.3602 - accuracy: 0.8484 - val_loss: 0.3505 - val_accuracy: 0.8398
Epoch 7/20
93/93 [==============================] - 0s 3ms/step - loss: 0.3450 - accuracy: 0.8491 - val_loss: 0.3404 - val_accuracy: 0.8412
Epoch 8/20
93/93 [==

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Load your merged_df dataset and preprocess it (as you've done)
# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 0 for Healthy Control and 1 for Alcoholic Liver Disease

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_classes = [0, 1]
X_filtered = X[y.isin(selected_classes)]
y_filtered = y[y.isin(selected_classes)]

# Split the filtered dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

# Standardize the input features (important for AdaBoost)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an AdaBoost classifier with a decision tree as the base estimator
adaboost_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)

# Train the AdaBoost classifier on your data
adaboost_classifier.fit(X_train, y_train)

# Predict using the trained AdaBoost classifier
y_pred = adaboost_classifier.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 84.59%


# **Chosen algorithm - xgboost**

In [44]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Load your merged_df dataset and preprocess it
# Replace 'your_merged_df.csv' with your dataset file path

# Create an empty list to store DataFrames
results_list = []

# Define features (X) and target variable (y) for classification
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # 0 for Healthy Control, 1 for Alcoholic Liver Disease, and 2 for Parkinson's

# Specify the number of iterations
num_iterations = 32

for iteration in range(num_iterations):
    # Split the dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier with specific hyperparameters
    xgb_classifier = xgb.XGBClassifier(
        objective='binary:logistic',
        random_state=42,
        n_estimators=100,  # Number of boosting rounds
        max_depth=6,       # Maximum tree depth
        learning_rate=0.1, # Learning rate
        subsample=0.8,     # Fraction of samples used for each tree
    )
    xgb_classifier.fit(X_train, y_train)

    # Predict disease labels on the test set
    y_pred = xgb_classifier.predict(X_test)

    # Calculate accuracy for each class
    accuracy_healthy_control = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
    accuracy_alcoholic_liver_disease = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])
    accuracy_parkinsons = accuracy_score(y_test[y_test == 2], y_pred[y_test == 2])

    # Create a DataFrame for the current iteration with model information
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Healthy Control Accuracy': [accuracy_healthy_control],
        'Alcoholic Liver Disease Accuracy': [accuracy_alcoholic_liver_disease],
        'Parkinson\'s Accuracy': [accuracy_parkinsons],
        'Model Info': [str(xgb_classifier)],
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_classification_results = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_classification_results.head(num_iterations)


,Iteration,Healthy Control Accuracy,Alcoholic Liver Disease Accuracy,Parkinson's Accuracy,Model Info
0,1,0.905473,0.820588,0.726644,"XGBClassifier(base_score=None, booster=None, c..."
1,2,0.898276,0.833819,0.676375,"XGBClassifier(base_score=None, booster=None, c..."
2,3,0.900974,0.800000,0.690722,"XGBClassifier(base_score=None, booster=None, c..."
3,4,0.911223,0.803519,0.704082,"XGBClassifier(base_score=None, booster=None, c..."
4,5,0.909710,0.850299,0.659164,"XGBClassifier(base_score=None, booster=None, c..."
5,6,0.898058,0.814815,0.627760,"XGBClassifier(base_score=None, booster=None, c..."
6,7,0.925801,0.827485,0.676768,"XGBClassifier(base_score=None, booster=None, c..."
7,8,0.899489,0.802469,0.697819,"XGBClassifier(base_score=None, booster=None, c..."
8,9,0.917763,0.849530,0.662295,"XGBClassifier(base_score=None, booster=None, c..."
9,10,0.925424,0.857143,0.608563,"XGBClassifier(base_score=None, booster=None, c..."


In [41]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Create an empty list to store DataFrames
results_list = []

# Define features (X) and target variable (y) for classification
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # 0 for Healthy Control, 1 for Alcoholic Liver Disease, and 2 for Parkinson's

# Specify the number of iterations
num_iterations = 32

for iteration in range(num_iterations):
    # Split the dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier with specific hyperparameters
    xgb_classifier = xgb.XGBClassifier(
        objective='multi:softmax',  # Multi-class classification
        num_class=3,                # Number of classes
        random_state=42,
        n_estimators=100,           # Number of boosting rounds
        max_depth=6,                # Maximum tree depth
        learning_rate=0.1,          # Learning rate
        subsample=0.8,              # Fraction of samples used for each tree
    )

    # Perform k-fold cross-validation
    cv_scores = cross_val_score(xgb_classifier, X_train, y_train, cv=5, scoring='accuracy')

    # Make predictions on the test set
    xgb_classifier.fit(X_train, y_train)
    y_pred = xgb_classifier.predict(X_test)

    # Calculate confusion matrix and classification report
    confusion = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, target_names=['Healthy Control', 'Alcoholic Liver Disease', 'Parkinson\'s'], output_dict=True)

    # Create a DataFrame for the current iteration with model information and evaluation results
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Cross-Validation Mean Accuracy': [cv_scores.mean()],
        'Cross-Validation Std Dev': [cv_scores.std()],
        'Confusion Matrix': [confusion],
        'Classification Report': [class_report],
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_classification_results = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_classification_results.head(50)


,Iteration,Cross-Validation Mean Accuracy,Cross-Validation Std Dev,Confusion Matrix,Classification Report
0,1,0.818997,0.006699,"[[546, 23, 34], [25, 279, 36], [71, 8, 210]]",{'Healthy Control': {'precision': 0.8504672897...
1,2,0.812903,0.009647,"[[521, 16, 43], [29, 286, 28], [83, 17, 209]]",{'Healthy Control': {'precision': 0.8230647709...
2,3,0.819194,0.006092,"[[555, 25, 36], [28, 260, 37], [75, 15, 201]]",{'Healthy Control': {'precision': 0.8434650455...
3,4,0.820210,0.007498,"[[544, 17, 36], [36, 274, 31], [69, 18, 207]]",{'Healthy Control': {'precision': 0.8382126348...
4,5,0.812296,0.005222,"[[534, 18, 35], [27, 284, 23], [89, 17, 205]]",{'Healthy Control': {'precision': 0.8215384615...
5,6,0.817573,0.007573,"[[555, 18, 45], [27, 242, 28], [99, 19, 199]]",{'Healthy Control': {'precision': 0.8149779735...
6,7,0.815546,0.011854,"[[549, 18, 26], [33, 283, 26], [78, 18, 201]]",{'Healthy Control': {'precision': 0.8318181818...
7,8,0.816764,0.011412,"[[528, 21, 38], [42, 260, 22], [75, 22, 224]]",{'Healthy Control': {'precision': 0.8186046511...
8,9,0.809656,0.009608,"[[558, 18, 32], [27, 271, 21], [85, 18, 202]]",{'Healthy Control': {'precision': 0.8328358208...
9,10,0.817981,0.007138,"[[546, 20, 24], [26, 270, 19], [99, 29, 199]]",{'Healthy Control': {'precision': 0.8137108792...


### xgBoost: HC vs. ALD Accuracy

In [19]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Load your merged_df dataset and preprocess it
# Replace 'your_merged_df.csv' with your dataset file path

# Create an empty list to store DataFrames
results_list = []

# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # 0 for Healthy Control, 1 for Alcoholic Liver Disease, and 2 for Parkinson's

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_groups = [0, 1]
X_filtered = X[y.isin(selected_groups)]
y_filtered = y[y.isin(selected_groups)]

# Specify the number of iterations
num_iterations = 32

for iteration in range(num_iterations):
    # Split the filtered dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier
    xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
    xgb_classifier.fit(X_train, y_train)

    # Predict disease labels on the test set
    y_pred = xgb_classifier.predict(X_test)

    # Calculate accuracy for Healthy Control (0) and Alcoholic Liver Disease (1)
    accuracy_healthy_control = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
    accuracy_alcoholic_liver_disease = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])

    # Create a DataFrame for the current iteration
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Healthy Control Accuracy': [accuracy_healthy_control],
        'Alcoholic Liver Disease Accuracy': [accuracy_alcoholic_liver_disease],
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_ald_accuracy = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_ald_accuracy.head(50)

,Iteration,Healthy Control Accuracy,Alcoholic Liver Disease Accuracy
0,1,0.959933,0.866261
1,2,0.940778,0.896142
2,3,0.944444,0.911932
3,4,0.964401,0.896774
4,5,0.948074,0.924471
5,6,0.972536,0.922330
6,7,0.955519,0.919003
7,8,0.947899,0.894895
8,9,0.951220,0.916933
9,10,0.958264,0.893617


In [20]:
xgboost_ald_accuracy.describe()

,Iteration,Healthy Control Accuracy,Alcoholic Liver Disease Accuracy
count,20.00000,20.000000,20.000000
mean,10.50000,0.956241,0.907194
std,5.91608,0.009537,0.018432
min,1.00000,0.940778,0.866261
25%,5.75000,0.949456,0.894575
50%,10.50000,0.954761,0.904037
75%,15.25000,0.964271,0.921434
max,20.00000,0.973510,0.940439


### xgBoost: HC vs. ALD Sensitivity

In [21]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Load your merged_df dataset and preprocess it
# Replace 'your_merged_df.csv' with your dataset file path

# Create an empty list to store DataFrames
results_list = []

# Define features (X) and target variable (y)
X = merged_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = merged_df['group']  # Assuming 0 for Healthy Control, 1 for Alcoholic Liver Disease, and 2 for Parkinson's

# Filter the dataset to include only "Healthy Control" (0) and "Alcoholic Liver Disease" (1)
selected_groups = [0, 1]
X_filtered = X[y.isin(selected_groups)]
y_filtered = y[y.isin(selected_groups)]

# Specify the number of iterations
num_iterations = 32

# Define a function to calculate sensitivity and specificity
def sensitivity_specificity(y_true, y_pred, positive_class=1):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    if positive_class == 1:
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
    else:
        sensitivity = tn / (tn + fp)
        specificity = tp / (tp + fn)
    return sensitivity, specificity

for iteration in range(num_iterations):
    # Split the filtered dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier
    xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
    xgb_classifier.fit(X_train, y_train)

    # Predict disease labels on the test set
    y_pred = xgb_classifier.predict(X_test)

    # Calculate sensitivity and specificity for Healthy Control (0) and Alcoholic Liver Disease (1)
    sensitivity_healthy_control, specificity_healthy_control = sensitivity_specificity(y_test, y_pred, positive_class=0)
    sensitivity_alcoholic_liver_disease, specificity_alcoholic_liver_disease = sensitivity_specificity(y_test, y_pred, positive_class=1)

    # Create a DataFrame for the current iteration
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Healthy Control Sensitivity': [sensitivity_healthy_control],
        'Alcoholic Liver Disease Sensitivity': [sensitivity_alcoholic_liver_disease],
        'Healthy Control Specificity': [specificity_healthy_control],
        'Alcoholic Liver Disease Specificity': [specificity_alcoholic_liver_disease]
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_ald_sen = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_ald_sen.head(num_iterations)


,Iteration,Healthy Control Sensitivity,Alcoholic Liver Disease Sensitivity,Healthy Control Specificity,Alcoholic Liver Disease Specificity
0,1,0.959933,0.866261,0.866261,0.959933
1,2,0.940778,0.896142,0.896142,0.940778
2,3,0.944444,0.911932,0.911932,0.944444
3,4,0.964401,0.896774,0.896774,0.964401
4,5,0.948074,0.924471,0.924471,0.948074
5,6,0.972536,0.922330,0.922330,0.972536
6,7,0.955519,0.919003,0.919003,0.955519
7,8,0.947899,0.894895,0.894895,0.947899
8,9,0.951220,0.916933,0.916933,0.951220
9,10,0.958264,0.893617,0.893617,0.958264


In [22]:
xgboost_ald_sen.describe()

,Iteration,Healthy Control Sensitivity,Alcoholic Liver Disease Sensitivity,Healthy Control Specificity,Alcoholic Liver Disease Specificity
count,20.00000,20.000000,20.000000,20.000000,20.000000
mean,10.50000,0.956241,0.907194,0.907194,0.956241
std,5.91608,0.009537,0.018432,0.018432,0.009537
min,1.00000,0.940778,0.866261,0.866261,0.940778
25%,5.75000,0.949456,0.894575,0.894575,0.949456
50%,10.50000,0.954761,0.904037,0.904037,0.954761
75%,15.25000,0.964271,0.921434,0.921434,0.964271
max,20.00000,0.973510,0.940439,0.940439,0.973510


## xgBoost: Control vs. Parkinson's Accuracy

In [23]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Load your merged_df dataset and preprocess it
# Replace 'your_merged_df.csv' with your dataset file path

# Create an empty list to store DataFrames
results_list = []

# Filter the dataset to include only Healthy Control (group 0) and Parkinson's (group 2)
filtered_df = merged_df[(merged_df['group'] == 0) | (merged_df['group'] == 2)]

# Define features (X) and the modified target variable (y)
X = filtered_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = filtered_df['group']  # Assuming 0 for Healthy Control and 2 for Parkinson's

# Map the classes to [0, 1]
class_mapping = {0: 0, 2: 1}  # Map Parkinson's (2) to 1
y = y.map(class_mapping)

# Specify the number of iterations
num_iterations = 32

for iteration in range(num_iterations):
    # Split the dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier
    xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
    xgb_classifier.fit(X_train, y_train)

    # Predict binary labels on the test set
    y_pred = xgb_classifier.predict(X_test)

    # Calculate accuracy for "Healthy Control" (0) and "Parkinson's" (1)
    accuracy_healthy_control = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
    accuracy_parkinsons = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])

    # Create a DataFrame for the current iteration
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Healthy Control Accuracy': [accuracy_healthy_control],
        'Parkinson\'s Accuracy': [accuracy_parkinsons],
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_pd_accuracy = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_pd_accuracy.head(num_iterations)


,Iteration,Healthy Control Accuracy,Parkinson's Accuracy
0,1,0.906667,0.822368
1,2,0.905724,0.822581
2,3,0.890344,0.849829
3,4,0.932998,0.846906
4,5,0.922442,0.802013
5,6,0.918333,0.779605
6,7,0.913115,0.792517
7,8,0.942470,0.741214
8,9,0.900974,0.847222
9,10,0.912480,0.815331


In [24]:
xgboost_pd_accuracy.describe()

,Iteration,Healthy Control Accuracy,Parkinson's Accuracy
count,20.00000,20.000000,20.000000
mean,10.50000,0.918530,0.805424
std,5.91608,0.012449,0.027870
min,1.00000,0.890344,0.741214
25%,5.75000,0.911322,0.789356
50%,10.50000,0.921098,0.806461
75%,15.25000,0.924934,0.822421
max,20.00000,0.942470,0.849829


In [25]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix

# Suppress XGBoost-related warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')

# Load your merged_df dataset and preprocess it (as you've done)

# Create an empty list to store DataFrames
results_list = []

# Filter the dataset to include only Healthy Control (group 0) and Parkinson's (group 2)
filtered_df = merged_df[(merged_df['group'] == 0) | (merged_df['group'] == 2)]

# Define features (X) and the modified target variable (y)
X = filtered_df[['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
              'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
              'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32']]
y = filtered_df['group']  # Assuming 0 for Healthy Control and 2 for Parkinson's

# Map the classes to [0, 1]
class_mapping = {0: 0, 2: 1}  # Map Parkinson's (2) to 1
y = y.map(class_mapping)

# Specify the number of iterations
num_iterations = 20

for iteration in range(num_iterations):
    # Split the dataset into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=iteration)

    # Create and train the XGBoost classifier
    xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
    xgb_classifier.fit(X_train, y_train)

    # Predict binary labels on the test set
    y_pred = xgb_classifier.predict(X_test)

    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Calculate sensitivity (True Positive Rate) for "Parkinson's" (1)
    sensitivity_parkinsons = cm[1, 1] / (cm[1, 0] + cm[1, 1])

    # Calculate specificity (True Negative Rate) for "Healthy Control" (0)
    specificity_healthy_control = cm[0, 0] / (cm[0, 0] + cm[0, 1])

    # Create a DataFrame for the current iteration
    iteration_df = pd.DataFrame({
        'Iteration': [iteration + 1],
        'Parkinson\'s Sensitivity': [sensitivity_parkinsons],
        'Healthy Control Specificity': [specificity_healthy_control],
    })

    # Append the DataFrame to the results list
    results_list.append(iteration_df)

# Concatenate all DataFrames in the results list
xgboost_pd_sen_spec = pd.concat(results_list, ignore_index=True)

# Display the results DataFrame
xgboost_pd_sen_spec.head(num_iterations)


,Iteration,Parkinson's Sensitivity,Healthy Control Specificity
0,1,0.822368,0.906667
1,2,0.822581,0.905724
2,3,0.849829,0.890344
3,4,0.846906,0.932998
4,5,0.802013,0.922442
5,6,0.779605,0.918333
6,7,0.792517,0.913115
7,8,0.741214,0.942470
8,9,0.847222,0.900974
9,10,0.815331,0.912480


In [26]:
xgboost_pd_sen_spec.describe()

,Iteration,Parkinson's Sensitivity,Healthy Control Specificity
count,20.00000,20.000000,20.000000
mean,10.50000,0.805424,0.918530
std,5.91608,0.027870,0.012449
min,1.00000,0.741214,0.890344
25%,5.75000,0.789356,0.911322
50%,10.50000,0.806461,0.921098
75%,15.25000,0.822421,0.924934
max,20.00000,0.849829,0.942470
